In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

In [2]:
df=pd.read_csv('finalPSL.csv')

In [3]:
df.head()

,team_1,team_2,total_runs,wickets_left,balls_left,crr,last_five,Target
0,Quetta Gladiators,Islamabad United,27,8,100,6.230769,26.0,156
1,Lahore Qalandars,Quetta Gladiators,12,10,108,4.000000,30.0,209
2,Multan Sultans,Lahore Qalandars,1,10,116,0.600000,41.0,200
3,Lahore Qalandars,Peshawar Zalmi,92,8,43,6.650602,65.0,164
4,Karachi Kings,Lahore Qalandars,107,7,54,8.916667,46.0,186


In [4]:
df['team_1'].values

array(['Quetta Gladiators', 'Lahore Qalandars', 'Multan Sultans', ...,
       'Karachi Kings', 'Multan Sultans', 'Lahore Qalandars'],
      dtype=object)

In [4]:
df.columns

Index(['team_1', 'team_2', 'total_runs', 'wickets_left', 'balls_left', 'crr',
       'last_five', 'Target'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19689 entries, 0 to 19688
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   team_1        19689 non-null  object 
 1   team_2        19689 non-null  object 
 2   total_runs    19689 non-null  int64  
 3   wickets_left  19689 non-null  int64  
 4   balls_left    19689 non-null  int64  
 5   crr           19689 non-null  float64
 6   last_five     19689 non-null  float64
 7   Target        19689 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 1.2+ MB


In [6]:
X=df.drop(columns='Target')
Y=df['Target']
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.1,random_state=1)

In [7]:
X_train.shape

(17720, 7)

In [8]:
X_test.shape

(1969, 7)

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.model_selection import cross_val_score

In [10]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['team_1','team_2'])
]
,remainder='passthrough')

In [23]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=80,learning_rate=0.1,max_depth=12,random_state=1, subsample=0.6))
])

In [24]:
pipe.fit(X_train,Y_train)
y_pred = pipe.predict(X_test)
print(r2_score(Y_test,y_pred))
print(mean_absolute_error(Y_test,y_pred))

0.9143365264155252
5.189496353715128


In [25]:
#OVERFITTING
training_predictions = pipe.predict(X_train)
r2_score(Y_train,training_predictions)

0.986428246858474

In [26]:
cv_scores = cross_val_score(pipe,X,Y,cv=5, scoring='r2')

print("Cross-Validation Scores:", cv_scores)
print("Mean Cross-Validation Score:", cv_scores.mean())

Cross-Validation Scores: [0.90234827 0.90366744 0.89777606 0.8915531  0.91281248]
Mean Cross-Validation Score: 0.9016314706977709


In [27]:
features = np.array([['Lahore Qalandars','Multan Sultans',85,9,60,8.5,52]])

features_df =pd.DataFrame(features,columns=['team_1', 'team_2', 'total_runs', 'wickets_left', 'balls_left', 'crr',
       'last_five'])

print('Predicted Score Is : ',pipe.predict(features_df))
print('Actural Score Is : ', 175)

Predicted Score Is :  [179.30212]
Actural Score Is :  175


In [44]:
features = np.array([['Peshawar Zalmi','Karachi Kings',116,8,48,9.66,52]])

features_df =pd.DataFrame(features,columns=['team_1', 'team_2', 'total_runs', 'wickets_left', 'balls_left', 'crr',
       'last_five'])

print('Predicted Score is : ',pipe.predict(features_df))
print('Actual Score is : 199')

Predicted Score is :  [192.35803]
Actual Score is : 199


In [46]:
features = np.array([['Quetta Gladiators','Multan Sultans',46,5,72,5.75,23]])

features_df =pd.DataFrame(features,columns=['team_1', 'team_2', 'total_runs', 'wickets_left', 'balls_left', 'crr',
       'last_five'])

print('Predicted Score is : ',pipe.predict(features_df))
print('Actual Score is : 110')

Predicted Score is :  [137.37257]
Actual Score is : 110


In [28]:
import pickle
#pickle.dump(pipe,open('pipe.pkl','wb'))